In [1]:
%cd /kaggle/working

import os

from hydra import compose, initialize
from omegaconf import OmegaConf

with initialize(
    version_base=None, config_path="../experiments/091_cloud_wather_then_q23"
):
    cfg = compose(
        config_name="config.yaml",
        overrides=["exp=base_all", "debug=True"],
        return_hydra_config=True,
    )

/kaggle/working


In [2]:
import warnings
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import polars as pl
import seaborn as sns

warnings.filterwarnings("ignore", "is_categorical_dtype")
warnings.filterwarnings("ignore", "use_inf_as_na")
pl.Config.set_tbl_cols(-1)
pl.Config.set_tbl_rows(100)

polars.config.Config

In [3]:
df = pl.read_parquet("input/train.parquet", n_rows=50000)
df = df[:, 1:]  # id を抜く

In [4]:
import xarray as xr

grid_path = "/kaggle/working/misc/grid_info/ClimSim_low-res_grid-info.nc"
grid_info = xr.open_dataset(grid_path)
hyam = grid_info["hyam"].to_numpy()
hybm = grid_info["hybm"].to_numpy()
hyai = grid_info["hyai"].to_numpy()
hybi = grid_info["hybi"].to_numpy()

P0 = 1e5

In [6]:
temperature_array = df[:, :60].to_numpy()
specific_humidity_array = df[:, 60:120].to_numpy()
near_surface_air_pressure = df[:, 360].to_numpy()
near_surface_air_pressure[0]

101135.6163329346

array([178.61875689, 187.11101102, 177.7739451 , 187.11493334,
       182.29315816, 187.5466503 , 185.24974843, 180.16120985,
       187.1654688 , 175.32836426, 174.65403678, 175.85836051,
       181.42510951, 188.92245675])

In [50]:
def eliq(T, method="paper"):
    """
    Function taking temperature (in K) and outputting liquid saturation
    pressure (in hPa) using a polynomial fit
    """
    if method == "paper":
        a_liq = np.array(
            [
                -0.976195544e-15,
                -0.952447341e-13,
                0.640689451e-10,
                0.206739458e-7,
                0.302950461e-5,
                0.264847430e-3,
                0.142986287e-1,
                0.443987641,
                6.11239921,
            ]
        )
        c_liq = -80
        T0 = 273.16
        return 100 * np.polyval(a_liq, np.maximum(c_liq, T - T0))

    elif method == "MurphyKoop":
        es = np.exp(
            54.842763
            - (6763.22 / T)
            - (4.210 * np.log(T))
            + (0.000367 * T)
            + (
                np.tanh(0.0415 * (T - 218.8))
                * (53.878 - (1331.22 / T) - (9.44523 * np.log(T)) + 0.014025 * T)
            )
        )
        return es
    elif method == "GoffGratch":
        tboil = 373.15  # Boiling point of water in Kelvin
        es = (
            10.0
            ** (
                -7.90298 * (tboil / T - 1.0)
                + 5.02808 * np.log10(tboil / T)
                - 1.3816e-7 * (10.0 ** (11.344 * (1.0 - T / tboil)) - 1.0)
                + 8.1328e-3 * (10.0 ** (-3.49149 * (tboil / T - 1.0)) - 1.0)
                + np.log10(1013.246)
            )
            * 100.0
        )
        return es
    elif method == "OldGoffGratch":
        tboil = 373.15  # Boiling point of water in Kelvin
        ps = 1013.246
        e1 = 11.344 * (1.0 - T / tboil)
        e2 = -3.49149 * (tboil / T - 1.0)
        f1 = -7.90298 * (tboil / T - 1.0)
        f2 = 5.02808 * np.log10(tboil / T)
        f3 = -1.3816 * (10.0**e1 - 1.0) / 10000000.0
        f4 = 8.1328 * (10.0**e2 - 1.0) / 1000.0
        f5 = np.log10(ps)
        f = f1 + f2 + f3 + f4 + f5
        es = (10.0**f) * 100.0
        return es
    elif method == "e3sm":
        a0 = 6.105851
        a1 = 0.4440316
        a2 = 0.1430341e-1
        a3 = 0.2641412e-3
        a4 = 0.2995057e-5
        a5 = 0.2031998e-7
        a6 = 0.6936113e-10
        a7 = 0.2564861e-13
        a8 = -0.3704404e-15

        dtt = T - 273.16
        esatw = a0 + dtt * (
            a1
            + dtt
            * (
                a2
                + dtt
                * (a3 + dtt * (a4 + dtt * (a5 + dtt * (a6 + dtt * (a7 + a8 * dtt)))))
            )
        )

        index = dtt <= -80.0
        esatw[index] = (
            2.0
            * 0.01
            * np.exp(9.550426 - 5723.265 / T + 3.53068 * np.log(T) - 0.00728332 * T)
        )[index]
        return esatw * (10**2)
    elif method == "Bolton":
        c1 = 611.2
        c2 = 17.67
        c3 = 243.5
        tmelt = 273.15  # Melting point of water in Kelvin
        es = c1 * np.exp((c2 * (T - tmelt)) / ((T - tmelt) + c3))
        return es
    elif method == "tenten":
        # https://metview.readthedocs.io/en/latest/api/functions/saturation_vapour_pressure.html
        a1 = 611.21
        a3 = 17.502
        a4 = 32.19
        return a1 * np.exp(a3 * (T - 273.16) / (T - a4))


def eice(T, method="paper"):
    """
    Function taking temperature (in K) and outputting ice saturation
    pressure (in hPa) using a polynomial fit
    """
    if method == "paper":
        a_ice = np.array(
            [
                0.252751365e-14,
                0.146898966e-11,
                0.385852041e-9,
                0.602588177e-7,
                0.615021634e-5,
                0.420895665e-3,
                0.188439774e-1,
                0.503160820,
                6.11147274,
            ]
        )
        c_ice = np.array([273.15, 185, -100, 0.00763685, 0.000151069, 7.48215e-07])
        T0 = 273.16
        return (
            (T > c_ice[0]) * eliq(T, method)
            + (T <= c_ice[0]) * (T > c_ice[1]) * 100 * np.polyval(a_ice, T - T0)
            + (T <= c_ice[1])
            * 100
            * (
                c_ice[3]
                + np.maximum(c_ice[2], T - T0)
                * (c_ice[4] + np.maximum(c_ice[2], T - T0) * c_ice[5])
            )
        )
    elif method == "GoffGratch":
        h2otrip = 273.16  # H2O triple point temperature in Kelvin
        es = (
            10.0
            ** (
                -9.09718 * (h2otrip / T - 1.0)
                - 3.56654 * np.log10(h2otrip / T)
                + 0.876793 * (1.0 - T / h2otrip)
                + np.log10(6.1071)
            )
            * 100.0
        )
        return es
    elif method == "OldGoffGratch":
        tmelt = 273.15  # Melting point of ice in Kelvin
        term1 = 2.01889049 / (tmelt / T)
        term2 = 3.56654 * np.log(tmelt / T)
        term3 = 20.947031 * (tmelt / T)
        es = 575.185606e10 * np.exp(-(term1 + term2 + term3))
        return es
    elif method == "MurphyKoop":
        es = np.exp(
            9.550426 - (5723.265 / T) + (3.53068 * np.log(T)) - (0.00728332 * T)
        )
        return es
    elif method == "e3sm":
        a0 = 6.11147274
        a1 = 0.503160820
        a2 = 0.188439774e-1
        a3 = 0.420895665e-3
        a4 = 0.615021634e-5
        a5 = 0.602588177e-7
        a6 = 0.385852041e-9
        a7 = 0.146898966e-11
        a8 = 0.252751365e-14

        dtt = T - 273.16
        esati = a0 + dtt * (
            a1
            + dtt
            * (
                a2
                + dtt
                * (a3 + dtt * (a4 + dtt * (a5 + dtt * (a6 + dtt * (a7 + a8 * dtt)))))
            )
        )

        index = dtt <= -80.0
        esati[index] = (
            0.01
            * np.exp(9.550426 - 5723.265 / T + 3.53068 * np.log(T) - 0.00728332 * T)
        )[index]
        return esati * (10**2)
    elif method == "Bolton":
        c1 = 611.2
        c2 = 17.67
        c3 = 243.5
        tmelt = 273.15  # Melting point of water in Kelvin
        es = c1 * np.exp((c2 * (T - tmelt)) / ((T - tmelt) + c3))
        return es
    elif method == "tenten":
        # https://metview.readthedocs.io/en/latest/api/functions/saturation_vapour_pressure.html
        a1 = 611.21
        a3 = 22.587
        a4 = -0.7
        return a1 * np.exp(a3 * (T - 273.16) / (T - a4))

In [51]:
def cal_specific2relative_coef(
    temperature_array,
    near_surface_air_pressure,
    hyam,
    hybm,
    method="paper",
):
    """
    specific humidity を relative humidity に変換するための係数を算出する（逆数を取れば逆変換にも使える）
    """
    P0 = 1e5  # Mean surface air pressure (Pa)
    # Formula to calculate air pressure (in Pa) using the hybrid vertical grid
    # coefficients at the middle of each vertical level: hyam and hybm
    air_pressure_Pa = hyam * P0 + hybm[None, :] * near_surface_air_pressure[:, None]

    # 1) Calculating saturation water vapor pressure
    T0 = 273.16  # Freezing temperature in standard conditions
    T00 = 253.16  # Temperature below which we use e_ice
    omega = (temperature_array - T00) / (T0 - T00)
    omega = np.maximum(0, np.minimum(1, omega))

    esat = omega * eliq(temperature_array, method) + (1 - omega) * eice(
        temperature_array, method
    )
    # 2) Calculating relative humidity
    Rd = 287  # Specific gas constant for dry air
    Rv = 461  # Specific gas constant for water vapor

    # We use the `values` method to convert Xarray DataArray into Numpy ND-Arrays
    return Rv / Rd * air_pressure_Pa / esat

In [29]:
t = np.array([178])
method = "MurphyKoop"
eliq(t, method), eice(t, method)

(array([0.00776218]), array([0.00368325]))

In [36]:
eice(t, "e3sm")

array([0.00368325])

In [42]:
specific2relative_coef = cal_specific2relative_coef(
    temperature_array,
    near_surface_air_pressure,
    hyam,
    hybm,
    method="paper",
)

relative_humidity = specific2relative_coef * specific_humidity_array
relative_humidity[relative_humidity > 3]

array([ 5.36684856,  5.04147533,  3.63954417,  3.19565073,  4.59619028,
        7.8351823 ,  6.70890873,  3.32977537,  5.32169224,  3.35840091,
        3.20610086, 11.34993881, 13.35702857,  3.7810839 ])

In [43]:
specific2relative_coef = cal_specific2relative_coef(
    temperature_array,
    near_surface_air_pressure,
    hyam,
    hybm,
    method="MurphyKoop",
)

relative_humidity = specific2relative_coef * specific_humidity_array
relative_humidity[relative_humidity > 3]

array([ 5.45573322,  5.31320472,  3.5725659 ,  3.36827275,  4.85876798,
        8.34472985,  6.3769877 ,  3.51622947,  5.61714999, 10.02147409,
       14.20738108,  4.07494236,  3.04531109])

In [45]:
specific2relative_coef = cal_specific2relative_coef(
    temperature_array,
    near_surface_air_pressure,
    hyam,
    hybm,
    method="e3sm",
)

relative_humidity = specific2relative_coef * specific_humidity_array
relative_humidity[relative_humidity > 3]

array([ 5.45573322,  5.31320472,  3.5725659 ,  3.36827275,  4.85876798,
        8.34472985,  6.3769877 ,  3.51622947,  5.61714999, 10.02147409,
       14.20738108,  4.07494236,  3.04531109])

In [48]:
specific2relative_coef = cal_specific2relative_coef(
    temperature_array,
    near_surface_air_pressure,
    hyam,
    hybm,
    method="GoffGratch",
)

relative_humidity = specific2relative_coef * specific_humidity_array
relative_humidity[relative_humidity > 3]

array([ 5.47734044,  5.33209734,  3.58686767,  3.38024902,  4.87713317,
        8.37423878,  6.40020395,  3.529885  ,  5.63710961, 10.06258128,
       14.26167753,  4.08910377,  3.05542924])

In [49]:
specific2relative_coef = cal_specific2relative_coef(
    temperature_array,
    near_surface_air_pressure,
    hyam,
    hybm,
    method="OldGoffGratch",
)

relative_humidity = specific2relative_coef * specific_humidity_array
relative_humidity[relative_humidity > 3]

array([ 5.47047071,  5.32570549,  3.5823482 ,  3.37619702,  4.87113622,
        8.3642229 ,  6.39245657,  3.52549466,  5.63035404, 10.04976817,
       14.24405962,  4.0842478 ,  3.05186639])

In [52]:
specific2relative_coef = cal_specific2relative_coef(
    temperature_array,
    near_surface_air_pressure,
    hyam,
    hybm,
    method="Bolton",
)

relative_humidity = specific2relative_coef * specific_humidity_array
relative_humidity[relative_humidity > 3]

array([4.19382676, 3.19561186, 5.07746685, 7.11752223])